In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [6]:
test_data = pd.read_csv('data/test_data_annot.csv')

In [7]:
test_data.head(n=10)

,chr,start,end,seq1,seq2,label,Median_sequence,GC,length,DNase,ATAC,H3K27ac,H3K4me3,DeepSpCas9_harmony,DeepCRISPR_harmony,CRISPRedict_harmony,Ruleset2_harmony,hyena_score
0,chr2,48440797,48440893,CGCCGCGCCTCCTCCTCCACTTCCGGGTTC,TTCCCGGGAGCGTGTCTGGGTTTGGGGGCG,1,AGTGGAGGAGGAGGCGCGGCGGCGGCGGCGGCGGCGGCTGCGGTGG...,0.729167,96,123332.744881,452.250715,9706.735352,16793.294783,4.431776,0.719472,1.736863,3.536433,-2.889551
1,chr7,5072216,5072322,GTGACTGGTCCGTGGGCTAACTCAGGGCTG,GCAGAGAGGAAAGGGCGTGAAGAGGGGGCG,1,AGGGCTGCTCGGGAGACATAGGCGGGCCGTGGGGGCAGGTGAAGGG...,0.669811,106,1910.800738,69.262120,0.000000,1901.978917,4.398179,0.718165,3.907437,4.064390,-3.844674
2,chr7,5072216,5072322,GCAGAGAGGAAAGGGCGTGAAGAGGGGGCG,GTGACTGGTCCGTGGGCTAACTCAGGGCTG,0,AGGGCTGCTCGGGAGACATAGGCGGGCCGTGGGGGCAGGTGAAGGG...,0.669811,106,1910.800738,69.262120,0.000000,1901.978917,4.382854,0.718165,3.898184,4.076555,-3.844674
3,chr2,46541807,46541911,AGATGGGTGTGTTCCGGCATCGCCAGGCAG,ACGGAATACCTTTGAGGAACATTGGGGACC,1,CAGGCAGAGGGGCAGGGGTTGCCGCCTCGAGCACAGGCCAAGTTTC...,0.538462,104,30556.754856,58.236825,17334.260518,16806.170692,4.380051,0.716701,2.081634,3.428387,-3.699923
4,chr2,46541807,46541911,ACGGAATACCTTTGAGGAACATTGGGGACC,AGATGGGTGTGTTCCGGCATCGCCAGGCAG,0,CAGGCAGAGGGGCAGGGGTTGCCGCCTCGAGCACAGGCCAAGTTTC...,0.538462,104,30556.754856,58.236825,17334.260518,16806.170692,4.378488,0.716701,2.081103,3.414082,-3.699923
5,chr2,46541807,46541911,ACGGAATACCTTTGAGGAACATTGGGGACC,AGATGGGTGTGTTCCGGCATCGCCAGGCAG,0,CAGGCAGAGGGGCAGGGGTTGCCGCCTCGAGCACAGGCCAAGTTTC...,0.538462,104,30556.754856,58.236825,17334.260518,16806.170692,4.378488,0.716701,2.081103,3.414082,-3.699923
6,chr7,5607353,5607456,GATAGCAGCTGCGAGAAGACTGAGGGGAGG,CTGTCCCTGCAGCCTAGGATGCAAAGGGAG,1,CATCCTAGGCTGCAGGGACAGGGGCTGGGCAGAGGCTCGCGGACCT...,0.621359,103,3816.656758,67.342133,2301.324533,2534.978231,4.311299,0.713438,2.803987,2.622306,-3.937159
7,chr7,5607353,5607456,GATAGCAGCTGCGAGAAGACTGAGGGGAGG,CTGTCCCTGCAGCCTAGGATGCAAAGGGAG,1,CATCCTAGGCTGCAGGGACAGGGGCTGGGCAGAGGCTCGCGGACCT...,0.621359,103,3816.656758,67.342133,2301.324533,2534.978231,4.311299,0.713438,2.803987,2.622306,-3.937159
8,chr3,46372159,46372259,ATTGGCATCCAGTATGTGCCCTCGAGGCCT,AGACTGAGTTGCAGCCGGGCATGGTGGCTC,0,GAGGCCTCTTAATTATTACTGGCTTGCTCATAGTGCATGTTCTTTG...,0.440000,100,301.381396,14.872258,1183.182586,458.406610,4.285287,0.712808,3.871828,3.438448,-3.522517
9,chr7,5607305,5607456,GATAGCAGCTGCGAGAAGACTGAGGGGAGG,CCTGACTGTGGCCTTGGTAAAGTCTGGAAA,1,TTTACCAAGGCCACAGTCAGGGCGACCTGGGTTCTGAGCCTCCCTT...,0.609272,151,3745.180017,68.042053,2349.736705,2443.449301,4.215512,0.708976,2.957499,2.796193,-3.006579


In [8]:
xgb_model = xgb.XGBRegressor()
xgb_model.load_model('./xgboost_model.model')

test_data = test_data.fillna(0.0)

features = ['DNase','ATAC','H3K27ac','H3K4me3','GC','length',
             'DeepSpCas9_harmony','DeepCRISPR_harmony','Ruleset2_harmony','CRISPRedict_harmony','hyena_score']

result = xgb_model.predict(test_data[features].values)
test_data['DeepDC_score'] = result.flatten()
result = test_data.sort_values(by=['DeepDC_score','DeepSpCas9_harmony','Ruleset2_harmony'],ascending=False)
result = result.loc[:,['chr','start','end','seq1','seq2']+features+['DeepDC_score']]
result['DeepDC_score'] = (result['DeepDC_score']-result['DeepDC_score'].min())/(result['DeepDC_score'].max()-result['DeepDC_score'].min())

In [9]:
result.head(n=10)

,chr,start,end,seq1,seq2,DNase,ATAC,H3K27ac,H3K4me3,GC,length,DeepSpCas9_harmony,DeepCRISPR_harmony,Ruleset2_harmony,CRISPRedict_harmony,hyena_score,DeepDC_score
88,chrX,134460075,134460193,CGGGAAAGCCGAGAGGTTCGCCTGAGGGCT,TACCTAGTGAGCCTGCAAACTGGTAGGCGC,39573.700063,343.787531,5497.827222,5659.409813,0.813559,118,2.967860,0.647498,3.735984,4.060762,-1.921438,1.000000
89,chrX,134460075,134460193,CGGGAAAGCCGAGAGGTTCGCCTGAGGGCT,TACCTAGTGAGCCTGCAAACTGGTAGGCGC,39573.700063,343.787531,5497.827222,5659.409813,0.813559,118,2.967860,0.647498,3.735984,4.060762,-1.921438,1.000000
116,chrX,134460075,134460193,TACCTAGTGAGCCTGCAAACTGGTAGGCGC,CGGGAAAGCCGAGAGGTTCGCCTGAGGGCT,39573.700063,343.787531,5497.827222,5659.409813,0.813559,118,2.867705,0.647498,3.734626,4.049088,-1.921438,0.989467
49,chrX,134460075,134460265,GACTGCTCAGGAGGAGGAAGCCGGTGGCGG,TACCTAGTGAGCCTGCAAACTGGTAGGCGC,40528.749008,442.541320,8895.466583,9544.306015,0.789474,190,3.285896,0.662078,3.835394,4.168556,-0.478027,0.966732
50,chrX,134460075,134460265,GACTGCTCAGGAGGAGGAAGCCGGTGGCGG,TACCTAGTGAGCCTGCAAACTGGTAGGCGC,40528.749008,442.541320,8895.466583,9544.306015,0.789474,190,3.285896,0.662078,3.835394,4.168556,-0.478027,0.966732
210,chrX,134460087,134460169,CTGCAAACTGGTAGGCGCCGGCGTAGGCGC,CGGGGCCTGCTTCTCCTCAGCTTCAGGCGG,39241.287808,262.985563,4625.326268,4866.881647,0.829268,82,2.508752,0.635922,3.773248,1.502665,-2.860149,0.925245
458,chr2,47782830,47783003,TCCCGCTTCCGCTCCAGAGAGGCAGGGCTT,GCGGGGCTGGCACACTGGTGGGTAGGGGCG,39543.148190,228.834419,13247.822881,13497.759278,0.774566,173,1.987409,0.598211,3.172761,2.645223,-2.391154,0.920235
507,chrX,134460347,134460525,GCCCTGGCGTCGTGGTGAGCAGCTCGGCCT,GGCGGCAGGGAGGACGGAATGGCGGGGTTT,12290.492133,270.855801,36113.956146,34834.762318,0.780899,178,1.915029,0.593571,4.287336,3.421448,-1.373258,0.916021
209,chr3,37176544,37176673,GCCCTCCCTCCAGCTTGGCCCTAACGGCTC,CACTGGTCCTGGTTCGAAGAGCCTGGGCAG,32203.903878,244.534504,62747.314154,35925.690945,0.790698,129,2.519144,0.644785,3.086435,2.432372,-3.116413,0.899307
383,chrX,134460075,134460275,CGCGCGGGCTGACTGCTCAGGAGGAGGAAG,TACCTAGTGAGCCTGCAAACTGGTAGGCGC,39948.335588,420.414254,9528.557602,10204.967485,0.780000,200,2.111969,0.615048,3.983258,3.759066,-0.307609,0.895570
